In [1]:
from astropy.stats import sigma_clipped_stats
from photutils.datasets import load_star_image
from photutils.detection import DAOStarFinder
# from photutils.aperture import aperture_photometry, CircularAperture

from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

from pathlib import Path
from astropy.nddata import CCDData
from astropy.io import fits
from ccdproc import ImageFileCollection
import ccdproc as ccdp

from astropy.nddata import CCDData
from astropy.stats import mad_std
import astropy.units as u
from convenience_functions import show_image
from astropy.timeseries import BinnedTimeSeries

# suppress warnings is not recommended :I
import warnings
warnings.filterwarnings("ignore")

In [2]:
matplotlib.rcParams['figure.dpi'] = 200
matplotlib.rcParams['figure.figsize'] = (12, 12)

In [3]:
cc_com_reduced = './data/cc_com/reduced/'
tz_boo_reduced = './data/tz_boo/reduced/'

def get_fluxes(images_dir, image_dim=(3056, 3056), r=50.0):
    images = ImageFileCollection(images_dir)
    images = images.files_filtered(include_path=True)
    
    mask = np.ones(image_dim, dtype=bool)
    mask[1400:1600, 1400:1600] = False
    
    timestamps = []
    xcentroids = []
    ycentroids = []
    
    DAO_fluxes = []
    # PSF_fluxes = []
    for i, path in np.ndenumerate(images):
        # print(path)
        image = CCDData.read(path)
        mean, median, stdev = sigma_clipped_stats(image.data, sigma=3.0)
    
        daofind = DAOStarFinder(fwhm=3.0, threshold=5.0*stdev)
        sources = daofind(image.data - median, mask=mask)
        image.header['DAO-FLUX'] = sources[0]['flux']
        
        x_centroid = sources[0]['xcentroid']
        y_centroid = sources[0]['ycentroid']
        image.header['X-CENTROID'] = x_centroid
        image.header['Y-CENTROID'] = y_centroid
        xcentroids.append(x_centroid)
        ycentroids.append(y_centroid)
        
        timestamps.append(image.header['DATE-OBS'])
        
        # aperture_flux
        # position = np.transpose((
        #     x_centroid,
        #     y_centroid
        # ))
        # aperture = CircularAperture(position, r=r)
        # phot = aperture_photometry(image, aperture)[0]['aperture_sum']
        # image.header['APT-FLUX'] = phot.value
        
        # psf flux
        # im just gonna do this after
        
        
        
        image.write(path, overwrite=True)
        DAO_fluxes.append(sources[0]['flux'])
        
    light_curve = BinnedTimeSeries(
        time_bin_start=timestamps,
        time_bin_size=image.header['EXPTIME'] * u.s,
        data={
            'xcentroid' : xcentroids,
            'ycentroid' : ycentroids,
            'DAO flux' : DAO_fluxes * u.adu
            # 'APT flux' : APT_fluxes * u.adu
        }
    )
        
    return light_curve
    

In [4]:
tzboo_fluxes = get_fluxes(tz_boo_reduced)
tzboo_fluxes

time_bin_start,time_bin_size,xcentroid,ycentroid,DAO flux
,s,,,adu
Time,float64,float64,float64,float64
2023-05-05T03:37:43.990,15.0,1505.6580302009559,1451.9594233158805,45.245654889464376
2023-05-05T04:06:02.330,15.0,1486.5069797565393,1459.4520589287135,70.87017499068716
2023-05-05T04:33:47.270,15.0,1529.8080990654942,1527.4393652092494,128.62406166002282
2023-05-05T05:33:33.170,15.0,1524.2977804982177,1521.622146581077,102.79422276187766
2023-05-05T04:33:47.270,15.0,1573.3308702131615,1415.1527729842312,1.0357153477871464


In [12]:
tzboo_fluxes.write('./results/tzboo_DAOresults.csv', overwrite=True)

In [6]:
cccom_fluxes = get_fluxes(cc_com_reduced)
cccom_fluxes.write('./results/cccom_DAOresults.csv', overwrite=True)
cccom_fluxes

time_bin_start,time_bin_size,xcentroid,ycentroid,DAO flux
,s,,,adu
Time,float64,float64,float64,float64
2023-05-05T03:51:49.420,15.0,1553.4850846834115,1449.1653270408465,7.1750177950703575
2023-05-05T04:15:01.570,15.0,1551.4549216092585,1438.7615661326574,4.400289640705585
2023-05-05T04:58:12.580,15.0,1550.134644481072,1438.672922784119,5.941673779134911


In [ ]:
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils.aperture import CircularAperture

tzboo3 = CCDData.read('./data/tz_boo/reduced/REDUCED_TZ_Boo_15s_V_3_20230505_053321.fits')

mean, median, std = sigma_clipped_stats(tzboo3.data, sigma=3.0)
daofind = DAOStarFinder(fwhm=3.0, threshold=5.*std)
sources = daofind(tzboo3.data - median)

positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=50.0)
norm = ImageNormalize(stretch=SqrtStretch())
# plt.imshow(tzboo3.data, cmap='Greys', origin='lower', norm=norm,
#            interpolation='nearest');

fig, ax = plt.subplots(figsize=(12, 12))

show_image(tzboo3.data[1400:1600, 1400:1600], fig=fig, ax=ax)
apertures.plot(color='red', lw=10, alpha=0.5);
